In [2]:
import pandas as pd
import os

In [3]:
ruta_archivos = [os.path.join('historicos_cubo', ruta) for ruta in os.listdir('historicos_cubo')]
ruta_archivos = sorted(ruta_archivos, key=lambda x: int(x.split('_')[3]))

In [4]:
dfs = {'_'.join(ruta.split('_')[4: 6]): pd.read_excel(ruta) for ruta in ruta_archivos}

In [18]:
historico_cubo = {}
for fecha_df, df in dfs.items():
    gasto_indirecto_tavi = df.loc[66, '270-PROCEDIMIENTOS TAVI']
    historico_cubo[fecha_df] = [gasto_indirecto_tavi]

In [34]:
gastos_tavi = pd.DataFrame.from_dict(historico_cubo).transpose().reset_index()
gastos_tavi.columns = ['Fecha', 'Gasto_Indirecto']
gastos_tavi['Fecha'] = pd.to_datetime(gastos_tavi['Fecha'], format='%Y_%m')

In [39]:
gastos_tavi.to_excel('tavi_indirectos_historicos.xlsx', index=False)

PermissionError: [Errno 13] Permission denied: 'tavi_indirectos_historicos.xlsx'